In [1]:
## Get dependencies ##

import numpy as np
import string
import math
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
sys.path.append('..')
from GIR import *
import scipy as sp
import pickle
import time
import scipy as sp
from scipy import signal
from scipy.io.idl import readsav
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import glob
import requests
import ftplib
import io
import cmocean
from bs4 import BeautifulSoup
import urllib.request
from io import StringIO, BytesIO
from zipfile import ZipFile
from tqdm import tqdm
import seaborn as sn

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
    
## Matplotlib rcparams setup:

matplotlib.rcParams['font.family']='Helvetica'
matplotlib.rcParams['font.size']=11
# matplotlib.rcParams['font.weight']=400

matplotlib.rcParams['image.cmap']='cmo.ice'

matplotlib.rcParams['axes.prop_cycle']=matplotlib.cycler('color',['011936','FF7D00','225560','BFACAA','D72638','788C9B','A33F00','7CAAB0','685655','EB767C'])
matplotlib.rcParams['axes.formatter.limits']=-3,3
matplotlib.rcParams['axes.labelweight']=300

matplotlib.rcParams['legend.frameon']=False

matplotlib.rcParams['boxplot.whiskers']=(5,95)
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.medianprops.color']='black'

matplotlib.rcParams['errorbar.capsize']=5

matplotlib.rcParams['hist.bins']='auto'

plt.rcParams['pdf.fonttype'] = 42

%matplotlib inline

%load_ext line_profiler
%load_ext memory_profiler

In [2]:
## get a large ensemble of parameters:

gas_params = pd.read_csv('../Parameter_Sets/Complete_gas_cycle_params.csv',skiprows=1,index_col=0)

thermal_params = pd.DataFrame(index=['d','q'],columns=[1,2,3])

thermal_params.loc[:] = [[0.935535,7.610096,277.278176],[0.200511,0.385134,0.501116]]

In [3]:
N = 1000

indep = True

gas_keys = ['mem'+str(x) for x in np.arange(N)]

if indep:
    thermal_keys = gas_keys.copy()
else:
    thermal_keys = ['tmem'+str(x) for x in np.arange(N)]

gas_param_ensemble = pd.concat([gas_params]*N,axis=1,keys=gas_keys)

thermal_param_ensemble = pd.concat([thermal_params]*N,axis=1,keys=thermal_keys)

In [4]:
## get SSP emissions
from tools.RCMIP import *

/home/leachl/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
../tools/RCMIP.py:60: PerformanceWarning: indexing past lexsort depth may impact performance.
  GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_concs.values(),'RCMIP_concs_unit'] = RCMIP_concs.loc[('World','ssp245')].reindex(RCMIP_to_GIR_map_concs.keys()).loc[:,'Unit'].values#.loc[('World','ssp245',RCMIP_to_GIR_map_concs.keys()),'Unit'].values


In [5]:
scenarios = ['ssp'+x for x in ['119','126','245','370','370-lowNTCF-aerchemmip','370-lowNTCF-gidden','434','460','534-over','585']]
ssp_emms = pd.concat([RCMIP_to_GIR_input_emms(x) for x in scenarios],keys=['esm-'+x+'-allGHG' for x in scenarios],axis=1).interpolate()
ssp_emms -= ssp_emms.loc[1750] # emissions relative to 1750 values
ssp_forc = pd.concat([get_RCMIP_forc(x) for x in scenarios],keys=['esm-'+x+'-allGHG' for x in scenarios],axis=1).interpolate()
ssp_forc -= ssp_forc.loc[1750]

In [6]:
ne.evaluate("log(0/1)")

/home/leachl/miniconda3/lib/python3.7/site-packages/numexpr/expressions.py:199: RuntimeWarning: divide by zero encountered in log
  return ConstantNode(func(*[x.value for x in args]))


array(-inf)

In [78]:
def calculate_alpha(G,G_A,T,r,g0,g1,iirf100_max = False):

#     iirf100_val = r[...,0] + r[...,1] * (G-G_A) + r[...,2] * T + r[...,3] * G_A
#     iirf100_val = np.abs(iirf100_val)
#     if iirf100_max:
#         iirf100_val = (iirf100_val>iirf100_max) * iirf100_max + iirf100_val * (iirf100_val<iirf100_max)
#     alpha_val = g0 * np.sinh(iirf100_val / g1)

    iirf100_val = ne.evaluate("abs(r0 + rU * (G-G_A) + rT * T + rA * G_A)",{'r0':r[...,0],'rU':r[...,1],'rT':r[...,2],'rA':r[...,3],'G':G,'G_A':G_A,'T':T})
    if iirf100_max:
        iirf100_val = ne.evaluate("where(iirf100_val>iirf100_max,iirf100_max,iirf100_val)")
    alpha_val = ne.evaluate("g0 * sinh(iirf100_val / g1)")

    return alpha_val

def step_concentration(R_old,G_A_old,E,alpha,a,tau,PI_conc,emis2conc,dt=1):
    
#     decay_rate = dt/(alpha*tau)
#     decay_factor = np.exp( -decay_rate )
#     R_new = E * a * 1/decay_rate * ( 1. - decay_factor ) + R_old * decay_factor
#     G_A = np.sum(R_new,axis=-1)
#     C = PI_conc + emis2conc * (G_A + G_A_old) / 2

    decay_rate = ne.evaluate("dt/(alpha*tau)")
    decay_factor = ne.evaluate("exp(-decay_rate)")
    R_new = ne.evaluate("E * a / decay_rate * ( 1. - decay_factor ) + R_old * decay_factor")
    G_A = ne.evaluate("sum(R_new,axis=4)")
    C = ne.evaluate("PI_conc + emis2conc * (G_A + G_A_old) / 2")

    return C,R_new,G_A

def step_forcing(C,PI_conc,f):
    
    # if the logarithmic/sqrt term is undefined (ie. C is zero or negative), this contributes zero to the overall forcing. An exception will appear, however.

#     logforc = f[...,0] * np.log(C / PI_conc)
#     linforc = f[...,1] * ( C - PI_conc )
#     sqrtforc = f[...,2] * (np.sqrt(C) - np.sqrt(PI_conc))
#     logforc[np.isnan(logforc)] = 0
#     sqrtforc[np.isnan(sqrtforc)] = 0

    logforc = ne.evaluate("f1 * where( (C/PI_conc) <= 0, 0, log(C/PI_conc) )",{'f1':f[...,0],'C':C,'PI_conc':PI_conc})
    linforc = ne.evaluate("f2 * (C - PI_conc)",{'f2':f[...,1],'C':C,'PI_conc':PI_conc})
    sqrtforc = ne.evaluate("f3 * ( (sqrt( where(C<0 ,0 ,C ) ) - sqrt(PI_conc)) )",{'f3':f[...,2],'C':C,'PI_conc':PI_conc})

    RF = logforc + linforc + sqrtforc

    return RF

def step_temperature(S_old,F,q,d,dt=1):

#     decay_factor = np.exp(-dt/d)
#     S_new = q * F * ( 1 - decay_factor ) + S_old * decay_factor
#     T = np.sum(S_old + S_new,axis=-1) / 2
    
    decay_factor = ne.evaluate("exp(-dt/d)")
    S_new = ne.evaluate("q * F * (1 - decay_factor) + S_old * decay_factor")
    T = ne.evaluate("sum( (S_old + S_new)/2, axis=3 )")

    return S_new,T

In [79]:
def run_GIR( emissions_in = False , concentrations_in = False , forcing_in = False , gas_parameters = get_gas_parameter_defaults() , thermal_parameters = get_thermal_parameter_defaults() , show_run_info = True ):

    # Determine the number of scenario runs , parameter sets , gases , integration period, timesteps

    # There are 2 modes : emissions_driven , concentration_driven

    # The model will assume if both are given then emissions take priority

    if emissions_in is False: # check if concentration driven
        concentration_driven = True
        emissions_in = return_empty_emissions(concentrations_in,gases_in=concentrations_in.columns.levels[1])
        time_index = concentrations_in.index
    else: # otherwise emissions driven
        concentration_driven=False
        time_index = emissions_in.index

    [(dim_scenario,scen_names),(dim_gas_param,gas_set_names),(dim_thermal_param,thermal_set_names)]=[(x.size,list(x)) for x in [emissions_in.columns.levels[0],gas_parameters.columns.levels[0],thermal_parameters.columns.levels[0]]]
    gas_names = [x for x in gas_parameters.columns.levels[1] if '|' not in x]
    n_gas = len(gas_names)
    n_forc,forc_names = gas_parameters.columns.levels[1].size,list(gas_parameters.columns.levels[1])
    n_year = time_index.size

    ## map the concentrations onto the forcings (ie. so the correct indirect forcing parameters read the correct concentration arrays)
    gas_forc_map = [gas_names.index(forc_names[x].split('|')[0]) for x in np.arange(len(forc_names))]

    names_list = [scen_names,gas_set_names,thermal_set_names,gas_names]
    names_titles = ['Scenario','Gas cycle set','Thermal set','Gas name']
    forc_names_list = [scen_names,gas_set_names,thermal_set_names,forc_names]
    forc_names_titles = ['Scenario','Gas cycle set','Thermal set','Forcing component']

    timestep = np.append(np.diff(time_index),np.diff(time_index)[-1])

    # check if no dimensions are degenerate
    if (set(scen_names) != set(gas_set_names))&(set(scen_names) != set(thermal_set_names))&(set(gas_set_names) != set(thermal_set_names)):
        gas_shape, gas_slice = [1,dim_gas_param,1],gas_set_names
        therm_shape, therm_slice = [1,1,dim_thermal_param],thermal_set_names
    # check if all degenerate
    elif (set(scen_names) == set(gas_set_names))&(set(scen_names) == set(thermal_set_names)):
        gas_shape, gas_slice = [dim_scenario,1,1],scen_names
        therm_shape, therm_slice = [dim_scenario,1,1],scen_names
        dim_gas_param = 1
        dim_thermal_param = 1
        [x.pop(1) for x in [names_list,names_titles,forc_names_list,forc_names_titles]]
        [x.pop(1) for x in [names_list,names_titles,forc_names_list,forc_names_titles]]
    # check other possibilities
    else:
        if set(scen_names) == set(gas_set_names):
            gas_shape, gas_slice = [dim_scenario,1,1],scen_names
            therm_shape, therm_slice = [1,1,dim_thermal_param],thermal_set_names
            dim_gas_param = 1
            [x.pop(1) for x in [names_list,names_titles,forc_names_list,forc_names_titles]]
        elif set(scen_names) == set(thermal_set_names):
            gas_shape, gas_slice = [1,dim_gas_param,1],gas_set_names
            therm_shape, therm_slice = [dim_scenario,1,1],scen_names
            dim_thermal_param = 1
            [x.pop(2) for x in [names_list,names_titles,forc_names_list,forc_names_titles]]
        else:
            gas_shape, gas_slice = [1,dim_gas_param,1],gas_set_names
            therm_shape, therm_slice = [1,dim_gas_param,1],gas_set_names
            dim_thermal_param = 1
            [x.pop(2) for x in [names_list,names_titles,forc_names_list,forc_names_titles]]

    ## Reindex to align columns:

    emissions = emissions_in.reindex(scen_names,axis=1,level=0).reindex(gas_names,axis=1,level=1).values.T.reshape(dim_scenario,1,1,n_gas,n_year)

    if forcing_in is False:
        ext_forcing = np.zeros((dim_scenario,1,1,1,n_year))
    else:
        forcing_in = forcing_in.reindex(scen_names,axis=1,level=0)
        ext_forcing = forcing_in.loc[:,(scen_names,slice(None))].values.T.reshape(dim_scenario,1,1,1,n_year)

    if concentration_driven:
        concentrations_in = concentrations_in.reindex(scen_names,axis=1,level=0).reindex(gas_names,axis=1,level=1)

    gas_cycle_parameters = gas_parameters.reindex(gas_slice,axis=1,level=0).reindex(gas_names,axis=1,level=1)
    thermal_parameters = thermal_parameters.reindex(therm_slice,axis=1,level=0)

    ## get parameter arrays
    a,tau,r,PI_conc,emis2conc=[gas_cycle_parameters.loc[x].values.T.reshape(gas_shape+[n_gas,-1]) for x in [['a1','a2','a3','a4'],['tau1','tau2','tau3','tau4'],['r0','rC','rT','rA'],'PI_conc','emis2conc']]
    f = gas_parameters.reindex(forc_names,axis=1,level=1).loc['f1':'f3'].values.T.reshape(gas_shape+[n_forc,-1])
    d,q = [thermal_parameters.loc[x].values.T.reshape(therm_shape+[-1]) for x in ['d','q']]

    if show_run_info:
        print('Integrating ' + str(dim_scenario) + ' scenarios, ' + str(dim_gas_param) + ' gas cycle parameter sets, ' + str(dim_thermal_param) + ' independent thermal response parameter sets, over ' + str(list(emissions_in.columns.levels[1])) + ', between ' + str(time_index[0]) + ' and ' + str(time_index[-1]) + '...',flush=True)

    # Dimensions : [scenario, gas params, thermal params, gas, time, (gas/thermal pools)]

    g1 = np.sum( a * tau * ( 1. - ( 1. + 100/tau ) * np.exp(-100/tau) ), axis=-1 )
    g0 = ( np.sinh( np.sum( a * tau * ( 1. - np.exp(-100/tau) ) , axis=-1) / g1 ) )**(-1.)

    # Create appropriate shape variable arrays / calculate RF if concentration driven

    C = np.empty((dim_scenario,dim_gas_param,dim_thermal_param,n_gas,n_year))
    RF = np.empty((dim_scenario,dim_gas_param,dim_thermal_param,n_forc,n_year))
    T = np.empty((dim_scenario,dim_gas_param,dim_thermal_param,n_year))
    alpha = np.empty((dim_scenario,dim_gas_param,dim_thermal_param,n_gas,n_year))
    alpha[...,0] = calculate_alpha(G=0,G_A=0,T=0,r=r,g0=g0,g1=g1)

    if concentration_driven:

        diagnosed_emissions = np.empty((dim_scenario,dim_gas_param,dim_thermal_param,n_gas,n_year))
        C[:] = concentrations_in.reindex(scen_names,axis=1,level=0).reindex(gas_names,axis=1,level=1).values.T.reshape(dim_scenario,1,1,n_gas,n_year)
        C_end = np.empty(C.shape)
        RF[:] = step_forcing(C[...,gas_forc_map,:],PI_conc[...,gas_forc_map,:],f[...,np.newaxis,:])
        C_end[...,0] = C[...,0]*2 - PI_conc[...,0]
        diagnosed_emissions[...,0],R,G_A = unstep_concentration(R_old=np.zeros(a.shape),C=C_end[...,0],alpha=alpha[...,0,np.newaxis],a=a,tau=tau,PI_conc=PI_conc[...,0],emis2conc=emis2conc[...,0],dt=timestep[0])
        S,T[...,0] = step_temperature(S_old=np.zeros(d.shape),F=np.sum(RF[...,0],axis=-1)[...,np.newaxis]+ext_forcing[...,0],q=q,d=d,dt=timestep[0])
        for t in tqdm(np.arange(1,n_year),unit=' timestep',):
            G = np.sum(diagnosed_emissions,axis=-1)
            alpha[...,t] = calculate_alpha(G=G,G_A=G_A,T=np.sum(S,axis=-1)[...,np.newaxis],r=r,g0=g0,g1=g1)
            C_end[...,t] = C[...,t]*2 - C_end[...,t-1]
            diagnosed_emissions[...,t],R,G_A = unstep_concentration(R_old=R,C=C_end[...,t],alpha=alpha[...,t,np.newaxis],a=a,tau=tau,PI_conc=PI_conc[...,0],emis2conc=emis2conc[...,0],dt=timestep[t])
            S,T[...,t] = step_temperature(S_old=S,F=np.sum(RF[...,t],axis=-1)[...,np.newaxis]+ext_forcing[...,t],q=q,d=d,dt=timestep[t])

        C_out = concentrations_in
        E_out = pd.DataFrame(np.moveaxis(diagnosed_emissions,-1,0).reshape(diagnosed_emissions.shape[-1],-1),index = time_index,columns=pd.MultiIndex.from_product(names_list,names=names_titles))

    if not concentration_driven:
        G = np.cumsum(emissions,axis=-1)
        C[...,0],R,G_A = step_concentration(R_old = 0,G_A_old = 0,alpha=alpha[...,0,np.newaxis],E=emissions[...,0,np.newaxis],a=a,tau=tau,PI_conc=PI_conc[...,0],emis2conc=emis2conc[...,0],dt=timestep[0])
        RF[...,0] = step_forcing(C=C[...,gas_forc_map,0],PI_conc=PI_conc[...,gas_forc_map,0],f=f)
        S,T[...,0] = step_temperature(S_old=0,F=np.sum(RF[...,0],axis=-1)[...,np.newaxis]+ext_forcing[...,0],q=q,d=d,dt=timestep[0])

        for t in tqdm(np.arange(1,n_year),unit=' timestep',):
            alpha[...,t] = calculate_alpha(G=G[...,t-1],G_A=G_A,T=np.sum(S,axis=-1)[...,np.newaxis],r=r,g0=g0,g1=g1)
            C[...,t],R,G_A = step_concentration(R_old = R,G_A_old=G_A,alpha=alpha[...,t,np.newaxis],E=emissions[...,t,np.newaxis],a=a,tau=tau,PI_conc=PI_conc[...,0],emis2conc=emis2conc[...,0],dt=timestep[t])
            RF[...,t] = step_forcing(C=C[...,gas_forc_map,t],PI_conc=PI_conc[...,gas_forc_map,0],f=f)
            S,T[...,t] = step_temperature(S_old=S,F=np.sum(RF[...,t],axis=-1)[...,np.newaxis]+ext_forcing[...,t],q=q,d=d,dt=timestep[t])

        C_out = pd.DataFrame(np.moveaxis(C,-1,0).reshape(C.shape[-1],-1),index = time_index,columns=pd.MultiIndex.from_product(names_list,names=names_titles))
        E_out = emissions_in

    ext_forcing = np.zeros(np.sum(RF,axis=-2)[...,np.newaxis,:].shape) + ext_forcing
    RF = np.concatenate((RF,ext_forcing),axis=-2)
    RF = np.concatenate((RF,np.sum(RF,axis=-2)[...,np.newaxis,:]),axis=-2)

    alpha_out = pd.DataFrame(np.moveaxis(alpha,-1,0).reshape(alpha.shape[-1],-1),index = time_index,columns=pd.MultiIndex.from_product(names_list,names=names_titles))
    RF_out = pd.DataFrame(np.moveaxis(RF,-1,0).reshape(RF.shape[-1],-1),index = time_index,columns=pd.MultiIndex.from_product([x+['External','Total']*(x==forc_names_list[-1]) for x in forc_names_list],names=forc_names_titles))
    T_out = pd.DataFrame(np.moveaxis(T,-1,0).reshape(T.shape[-1],-1),index = time_index,columns=pd.MultiIndex.from_product(names_list[:-1],names=names_titles[:-1]))

    out_dict = {'C':C_out, \
                'RF':RF_out, \
                'T':T_out, \
                'alpha':alpha_out, \
                'Emissions':E_out , \
                'gas_parameters':gas_parameters , \
                'thermal parameters':thermal_parameters}

    for axis in [x for x in list(out_dict.keys())[:-2] if type(x)==pd.core.frame.DataFrame]:
        out_dict[axis].index = out_dict[axis].index.rename('Year')

    return out_dict


In [80]:
%lprun -f run_GIR run_GIR(emissions_in=ssp_emms.reindex(np.arange(1750,2101)),forcing_in=ssp_forc.reindex(np.arange(1750,2101)),gas_parameters=gas_param_ensemble,thermal_parameters=thermal_param_ensemble)

Integrating 10 scenarios, 1000 gas cycle parameter sets, 1 independent thermal response parameter sets, over ['bc', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'cf4', 'cfc11', 'cfc113', 'cfc114', 'cfc115', 'cfc12', 'ch2cl2', 'ch3ccl3', 'chcl3', 'co', 'halon1202', 'halon1211', 'halon1301', 'halon2402', 'hcfc141b', 'hcfc142b', 'hcfc22', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methyl_bromide', 'methyl_chloride', 'nf3', 'nh3', 'nitrous_oxide', 'nmvoc', 'nox', 'oc', 'sf6', 'so2', 'so2f2'], between 1750 and 2100...


100%|██████████| 350/350 [00:19<00:00, 17.99 timestep/s]


Timer unit: 1e-06 s

Total time: 21.8822 s
File: <ipython-input-79-a6aee183343d>
Function: run_GIR at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def run_GIR( emissions_in = False , concentrations_in = False , forcing_in = False , gas_parameters = get_gas_parameter_defaults() , thermal_parameters = get_thermal_parameter_defaults() , show_run_info = True ):
     2                                           
     3                                               # Determine the number of scenario runs , parameter sets , gases , integration period, timesteps
     4                                           
     5                                               # There are 2 modes : emissions_driven , concentration_driven
     6                                           
     7                                               # The model will assume if both are given then emissions take priority
     8                    

In [75]:
check = run_GIR(emissions_in=ssp_emms.reindex(['esm-ssp245-allGHG'],axis=1,level=0).reindex(np.arange(1750,2101)),forcing_in=ssp_forc.reindex(['esm-ssp245-allGHG'],axis=1,level=0).reindex(np.arange(1750,2101)),gas_parameters=gas_param_ensemble,thermal_parameters=thermal_param_ensemble)

Integrating 1 scenarios, 1000 gas cycle parameter sets, 1 independent thermal response parameter sets, over ['bc', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'cf4', 'cfc11', 'cfc113', 'cfc114', 'cfc115', 'cfc12', 'ch2cl2', 'ch3ccl3', 'chcl3', 'co', 'halon1202', 'halon1211', 'halon1301', 'halon2402', 'hcfc141b', 'hcfc142b', 'hcfc22', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methyl_bromide', 'methyl_chloride', 'nf3', 'nh3', 'nitrous_oxide', 'nmvoc', 'nox', 'oc', 'sf6', 'so2', 'so2f2'], between 1750 and 2100...


100%|██████████| 350/350 [00:02<00:00, 156.31 timestep/s]
